In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [80]:
data = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv').head(800000).sort_values(['state'])
data.head()

/var/folders/0x/j2sfsgp167vf2w83dyjt6_nm0000gn/T/ipykernel_11304/468132992.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
408774,021307,2020/10/30,AK,021307,CORDOVA COMMUNITY MEDICAL CENTER,PO BOX 160 - 602 CHASE AVENUE,CORDOVA,99574.0,Critical Access Hospitals,2080.0,...,NaN,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0
126981,021311,2022/09/23,AK,021311,KETCHIKAN MEDICAL CENTER,3100 TONGASS AVENUE,KETCHIKAN,99901.0,Critical Access Hospitals,2130.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
319757,020012,2022/02/11,AK,020012,FAIRBANKS MEMORIAL HOSPITAL,1650 COWLES STREET,FAIRBANKS,99701.0,Short Term,2090.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
334772,020024,2021/06/04,AK,020024,CENTRAL PENINSULA GENERAL HOSPITAL,250 HOSPITAL PLACE,SOLDOTNA,99669.0,Short Term,2120.0,...,NaN,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0
714320,022001,2022/11/25,AK,022001,ST ELIAS SPECIALTY HOSPITAL,4800 CORDOVA STREET,ANCHORAGE,99503.0,Long Term,2020.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0


In [81]:
len(data)

742253

In [82]:
def ifZero(x):
    if x < 0:
        return 0
    return x

In [83]:
clean = data
for col in clean.columns:
  if clean[col].dtype != 'float64':
    clean = clean.drop(col, axis=1)
clean = clean.drop(['zip', 'fips_code'], axis=1)
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
408774,13.0,13.0,13.0,-999999.0,-999999.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
126981,13.6,13.6,13.6,13.6,13.6,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319757,NaN,NaN,82.0,78.0,70.0,8.0,7.4,6.9,-999999.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334772,115.0,115.0,61.0,41.9,41.9,-999999.0,-999999.0,-999999.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
714320,33.7,33.7,33.7,33.7,33.7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206253,89.0,86.0,86.0,27.8,27.6,-999999.0,-999999.0,-999999.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7315,12.0,NaN,NaN,5.0,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99894,88.0,65.0,65.0,45.7,43.7,10.1,10.1,10.1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-999999.0,-999999.0
715811,16.0,16.0,16.0,4.6,4.6,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
clean = clean.dropna()
clean = clean.applymap(lambda x: ifZero(x))
clean

,total_beds_7_day_avg,all_adult_hospital_beds_7_day_avg,all_adult_hospital_inpatient_beds_7_day_avg,inpatient_beds_used_7_day_avg,all_adult_hospital_inpatient_bed_occupied_7_day_avg,inpatient_beds_used_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_adult_patients_hospitalized_confirmed_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg,...,previous_day_admission_pediatric_covid_confirmed_0_4_7_day_sum,previous_day_admission_pediatric_covid_confirmed_12_17_7_day_sum,previous_day_admission_pediatric_covid_confirmed_5_11_7_day_sum,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_sum
122225,13.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
182645,48.8,49.0,54.9,44.4,41.0,4.3,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
449942,93.0,93.0,54.0,39.9,39.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
696560,13.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57881,51.5,51.0,53.0,45.7,45.1,4.1,4.1,4.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37286,46.1,27.3,23.0,10.6,10.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732897,60.0,60.0,48.0,25.6,25.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539686,18.0,18.0,14.0,5.7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295460,46.1,27.3,23.0,11.6,11.6,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
output = clean['inpatient_beds_used_covid_7_day_sum'].values.reshape(-1, 1)
clean = clean.drop('inpatient_beds_used_covid_7_day_sum', axis=1)
'inpatient_beds_used_covid_7_day_sum' in clean

False

In [86]:
solve = []
maxTrainAccuracy = -1
maxTestAccuracy = -1
index = -1
for col in clean:
  input = clean[col].values.reshape(-1, 1)
  xtrain, xtest, ytrain, ytest = train_test_split(input, output, test_size=0.3, random_state=404)
  dtree = DecisionTreeClassifier(random_state=404)
  dtree.fit(xtrain, ytrain)
  y_pred = dtree.predict(xtest)
  trainAccuracy = accuracy_score(y_true = ytrain, y_pred = dtree.predict(xtrain))
  testAccuracy = accuracy_score(y_true = ytest, y_pred = y_pred)
  
  if testAccuracy > maxTestAccuracy:
    maxTrainAccuracy = trainAccuracy
    maxTestAccuracy = testAccuracy
    index = len(solve)
    
  solve.append({
    'col': col,
    'input': input,
    'output': output,
    'xtrain': xtrain,
    'xtest': xtest,
    'ytrain': ytrain,
    'ytest': ytest,
    'dtree': dtree,
    'y_pred': y_pred,
    'trainAccuracy': trainAccuracy,
    'testAccuracy': testAccuracy,
  })
  

In [87]:
print('name:',solve[index].get('col'))
print("index:",index)
print('max training accuracy',maxTrainAccuracy)
print('max test accuracy',maxTestAccuracy)

name: total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum
index: 25
max training accuracy 0.8714067914067914
max test accuracy 0.8612727978269306
